## Goal: Consolidate all information relating to pricing.

#### Inputs: 
    * Inventory Health report from Amazon (contains age and stock level per SKU)
    * CDGD sourcing spreadsheet (contains seasonality classification and min/max prices)
    
#### Outputs: 
    * For inventory > 365 days, spreadsheet with columns: 
        sku -- asin -- isbn -- seasonal min -- current price -- inventory qty -- inventory age -- 
    * For all other inventory, spreadsheet with columns: 
    sku -- asin -- isbn -- seasonal min -- current price -- inventory qty -- inventory age -- competitor pricing

In [1]:
import pandas as pd
import numpy as np
import pyisbn #module for converting ISBN-13 to ISBN-10 and vice versa


invhealth = pd.read_csv('data/inventory_health_aug19.csv',engine='python')
FBAinv = pd.read_csv('data/FBA_inventory_aug19.csv',engine='python')
CDGD_inv = pd.read_csv('data/CDGD_inventory_aug20.csv')
CDGD_products = pd.read_csv('data/CDGD_sourcing_pricing_aug20.csv')
inv = pd.read_csv('data/inventory_spreadsheet_aug19.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'inventory_health_aug19.csv'

In [2]:
print(invhealth.columns)

# Useful: 
# 'sku','asin'
# 'condition', 'your-price','sellable-quantity'
# inv-age-0-to-90-days', 'inv-age-91-to-180-days','inv-age-181-to-270-days', 'inv-age-271-to-365-days','inv-age-365-plus-days'
# 'lowest-afn-used-price',  'lowest-mfn-used-price'

Index(['snapshot-date', 'sku', 'fnsku', 'asin', 'product-name', 'condition',
       'sales-rank', 'product-group', 'total-quantity', 'sellable-quantity',
       'unsellable-quantity', 'inv-age-0-to-90-days', 'inv-age-91-to-180-days',
       'inv-age-181-to-270-days', 'inv-age-271-to-365-days',
       'inv-age-365-plus-days', 'units-shipped-last-24-hrs',
       'units-shipped-last-7-days', 'units-shipped-last-30-days',
       'units-shipped-last-90-days', 'units-shipped-last-180-days',
       'units-shipped-last-365-days', 'weeks-of-cover-t7',
       'weeks-of-cover-t30', 'weeks-of-cover-t90', 'weeks-of-cover-t180',
       'weeks-of-cover-t365', 'num-afn-new-sellers', 'num-afn-used-sellers',
       'currency', 'your-price', 'sales-price', 'lowest-afn-new-price',
       'lowest-afn-used-price', 'lowest-mfn-new-price',
       'lowest-mfn-used-price', 'qty-to-be-charged-ltsf-12-mo',
       'qty-in-long-term-storage-program', 'qty-with-removals-in-progress',
       'projected-ltsf-12-mo', '

In [3]:
print(FBAinv.columns)
# Useful:  
# Nothing more than invhealth!!

Index(['sku', 'fnsku', 'asin', 'product-name', 'condition', 'your-price',
       'mfn-listing-exists', 'mfn-fulfillable-quantity', 'afn-listing-exists',
       'afn-warehouse-quantity', 'afn-fulfillable-quantity',
       'afn-unsellable-quantity', 'afn-reserved-quantity',
       'afn-total-quantity', 'per-unit-volume', 'afn-inbound-working-quantity',
       'afn-inbound-shipped-quantity', 'afn-inbound-receiving-quantity',
       'afn-researching-quantity', 'afn-reserved-future-supply',
       'afn-future-supply-buyable'],
      dtype='object')


In [30]:
print(inv.columns)

Index(['Title or Item Name', 'ISBN', 'Order #', 'Unnamed: 3', 'Source',
       'Order Date', 'Item status', 'Notes', 'Item Cost', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')


In [45]:
# From list of SKUs, get their min/max price from spreadsheet
SKUs = FBAinv['sku'].to_list()
ASINs = FBAinv['asin'].to_list()

ISBNs = list()
for asin in ASINs:
    try: 
        ISBNs.append(pyisbn.convert(asin))
    except:
        ISBNs.append('UNKNOWN')


min_prices = 4.99*np.ones(len(ISBNs))
max_prices = 299.99*np.ones(len(ISBNs))

my_price = np.ones(len(ISBNs))
avg_buy_cost = np.ones(len(ISBNs))
dont_go_below = np.zeros(len(ISBNs))
sellable_quantity = np.ones(len(ISBNs))
lowest_prime = np.zeros(len(ISBNs))
lowest_FBM = np.zeros(len(ISBNs))
age_0_3mo = np.zeros(len(ISBNs))
age_3_6mo = np.zeros(len(ISBNs))
age_6_9mo = np.zeros(len(ISBNs))
age_9_12mo = np.zeros(len(ISBNs))
age_over12mo = np.zeros(len(ISBNs))

df = pd.DataFrame({
    'SKU': SKUs,
    'ASIN': ASINs,
    'ISBN': ISBNs,
    'min_price': min_prices,
    'max_price': max_prices,
    'my_price': my_price,
    'avg_buy_cost': avg_buy_cost,
    'dont_go_below': dont_go_below,
    'condition': condition,
    'sellable_quantity': sellable_quantity,
    'lowest_prime': lowest_prime,
    'lowest_FBM': lowest_FBM,
    'age_0_3mo': age_0_3mo,
    'age_3_6mo': age_3_6mo,
    'age_6_9mo': age_6_9mo,
    'age_9_12mo': age_9_12mo,
    'age_over12mo': age_over12mo
    })

#min_price
for sku in SKUs:
    #convert to isbn
    isbn = df.loc[df['SKU'] == sku]['ISBN'].to_list()[0]
    min_price = CDGD_inv.loc[CDGD_inv['ISBN-13'] == isbn]['Approved list price'].to_list()
    if len(min_price)>0:
        df.at[df.index[df['ISBN']==isbn], 'min_price'] = min_price[0]
    
    
# my_price
for sku in SKUs:
    asin = df.loc[df['SKU'] == sku]['ASIN'].to_list()[0]
    try: 
        my_price = invhealth.loc[invhealth['asin'] == asin]['your-price'].to_list()[0]
    except:
        my_price = 'NOT_FOUND'            
    df.at[df.index[df['ASIN']==asin], 'my_price'] = my_price

    
# avg_buy_cost
for sku in SKUs:
    #convert to isbn
    isbn = df.loc[df['SKU'] == sku]['ISBN'].to_list()[0]
    avg_buy_cost = inv.loc[inv['ISBN'] == isbn]['Item Cost']
    if len(avg_buy_cost)>0:
        df.at[df.index[df['ISBN']==isbn], 'avg_buy_cost'] = round(np.mean(avg_buy_cost),2)
        
#dont_go_below
for sku in SKUs:
    #convert to isbn
    isbn = df.loc[df['SKU'] == sku]['ISBN'].to_list()[0]
    avg_buy_cost = inv.loc[inv['ISBN'] == isbn]['Item Cost']
    if len(avg_buy_cost)>0:
        df.at[df.index[df['ISBN']==isbn], 'dont_go_below'] = round(np.mean((avg_buy_cost)+10)*1.25,2)


#condition
for sku in SKUs:
    asin = df.loc[df['SKU'] == sku]['ASIN'].to_list()[0]
    try: 
        condition = invhealth.loc[invhealth['asin'] == asin]['condition'].to_list()[0]
    except:
        condition = 'NOT_FOUND'            
    df.at[df.index[df['ASIN']==asin], 'condition'] = condition
    
    

#sellable_quantity
for sku in SKUs:
    asin = df.loc[df['SKU'] == sku]['ASIN'].to_list()[0]
    try: 
        sellable_quantity = invhealth.loc[invhealth['asin'] == asin]['sellable-quantity'].to_list()[0]
    except:
        sellable_quantity = 'NOT_FOUND'            
    df.at[df.index[df['ASIN']==asin], 'sellable_quantity'] = sellable_quantity

#lowest_prime
for sku in SKUs:
    asin = df.loc[df['SKU'] == sku]['ASIN'].to_list()[0]
    try: 
        lowest_prime = invhealth.loc[invhealth['asin'] == asin]['lowest-afn-used-price'].to_list()[0]
    except:
        lowest_prime = 'NOT_FOUND'            
    df.at[df.index[df['ASIN']==asin], 'lowest_prime'] = lowest_prime

    
    
#lowest_FBM
for sku in SKUs:
    asin = df.loc[df['SKU'] == sku]['ASIN'].to_list()[0]
    try: 
        lowest_FBM = invhealth.loc[invhealth['asin'] == asin]['lowest-mfn-used-price'].to_list()[0]
    except:
        lowest_FBM = 'NOT_FOUND'            
    df.at[df.index[df['ASIN']==asin], 'lowest_FBM'] = lowest_FBM
    
    
#age_0_3mo
for sku in SKUs:
    asin = df.loc[df['SKU'] == sku]['ASIN'].to_list()[0]
    try: 
        age_0_3mo = invhealth.loc[invhealth['asin'] == asin]['inv-age-0-to-90-days'].to_list()[0]
    except:
        age_0_3mo = 'NOT_FOUND'            
    df.at[df.index[df['ASIN']==asin], 'age_0_3mo'] = age_0_3mo

    
#age_3_6mo
for sku in SKUs:
    asin = df.loc[df['SKU'] == sku]['ASIN'].to_list()[0]
    try: 
        age_3_6mo = invhealth.loc[invhealth['asin'] == asin]['inv-age-91-to-180-days'].to_list()[0]
    except:
        age_3_6mo = 'NOT_FOUND'            
    df.at[df.index[df['ASIN']==asin], 'age_3_6mo'] = age_3_6mo
 
    
#age_6_9mo
for sku in SKUs:
    asin = df.loc[df['SKU'] == sku]['ASIN'].to_list()[0]
    try: 
        age_6_9mo = invhealth.loc[invhealth['asin'] == asin]['inv-age-181-to-270-days'].to_list()[0]
    except:
        age_6_9mo = 'NOT_FOUND'            
    df.at[df.index[df['ASIN']==asin], 'age_6_9mo'] = age_6_9mo


#age_9_12mo

for sku in SKUs:
    asin = df.loc[df['SKU'] == sku]['ASIN'].to_list()[0]
    try: 
        age_9_12mo = invhealth.loc[invhealth['asin'] == asin]['inv-age-271-to-365-days'].to_list()[0]
    except:
        age_9_12mo = 'NOT_FOUND'            
    df.at[df.index[df['ASIN']==asin], 'age_9_12mo'] = age_9_12mo
    
    
#age_over12mo
for sku in SKUs:
    asin = df.loc[df['SKU'] == sku]['ASIN'].to_list()[0]
    try: 
        age_over12mo = invhealth.loc[invhealth['asin'] == asin]['inv-age-365-plus-days'].to_list()[0]
    except:
        age_over12mo = 'NOT_FOUND'            
    df.at[df.index[df['ASIN']==asin], 'age_over12mo'] = age_over12mo
                                                              
                                                              
# 'sku','asin'
# 'condition', 'your-price','sellable-quantity'
# inv-age-0-to-90-days', 'inv-age-91-to-180-days','inv-age-181-to-270-days', 'inv-age-271-to-365-days','inv-age-365-plus-days'
# 'lowest-afn-used-price',  'lowest-mfn-used-price'

In [46]:
df.to_csv(r'C:\Users\Lyra\Documents\GitHub\CDGD\df_allinfo_aug19.csv', index = False)